In [1]:
using DifferentialEquations
using ParameterizedFunctions
using Sundials
using DataFrames
using CSV

In [2]:
function temp_gen(hour, temp_ref)
    temp_ref + hour / (20 * 24 * 365) + 10 * sin((2 * pi / 24) * hour) + 10 * sin((2 * pi / (24 * 365)) * hour)
end

function I_S(hour)
    0.001 + 0.0005 * sin((2 * pi / (24 * 365)) * hour) #Exogenous SOC input
end

function I_D(hour)
    0.0001 + 0.00005 * sin((2 * pi / (24 * 365)) * hour) #Exogenous DOC input
end

function arrhenius_temp_dep(parameter, temp, Ea, temp_ref)
    decayed_parameter = parameter * exp(-Ea / 0.008314 * (1 / temp - 1 / temp_ref))
end

function linear_temp_dep(parameter, temp, Q, temp_ref)
    modified_parameter = parameter - Q * (temp - temp_ref)
end

function analytical_steady_state_AWB_full_ECA(SOC_input, DOC_input, u_Q_ref, Q, a_MSA, K_DE, K_UE, V_DE_ref, V_UE_ref, Ea_V_DE, Ea_V_UE, r_M, r_E, r_L, temp_ref)
    u_Q = linear_temp_dep(u_Q_ref, temp_ref, Q, temp_ref)
    V_DE = arrhenius_temp_dep(V_DE_ref, temp_ref, Ea_V_DE, temp_ref)
    V_UE = arrhenius_temp_dep(V_UE_ref, temp_ref, Ea_V_UE, temp_ref)    
    S₀ = ((-K_DE * r_L * (r_E + r_M) * (u_Q - 1) + r_E * u_Q * (SOC_input + DOC_input)) * (SOC_input * r_E * (u_Q - 1) - a_MSA * DOC_input * r_M * u_Q + SOC_input * r_M * (u_Q - a_MSA * u_Q - 1))) / ((r_E + r_M) * (u_Q - 1) * (DOC_input * u_Q * (r_E * V_DE - a_MSA * r_L * r_M) + SOC_input * (r_E * r_L * (u_Q - 1) + r_L * r_M * (u_Q - a_MSA * u_Q - 1) + r_E * u_Q * V_DE)))
    D₀ = -(K_UE * (r_E + r_M) * (u_Q - 1) - (SOC_input + DOC_input) * u_Q) / ((u_Q - 1) * (r_E + r_M - u_Q * V_UE))
    M₀ = -((SOC_input + DOC_input) * u_Q) / ((r_E + r_M) * (u_Q - 1))
    E₀ = r_E * M₀ / r_L
    #E₀ = -(((SOC_input + DOC_input) * r_E * r_Q) / (r_L * (r_E + r_M) * (u_Q - 1)))     
    return [S₀, D₀, M₀, E₀]    
end

analytical_steady_state_AWB_full_ECA (generic function with 1 method)

In [3]:
########################
###Non-param Constants##
########################

temp_ref = 283

#########################
##ODE System Parameters##
#########################

u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Separate parameters for full ECA
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50

50

In [4]:
###############
##ODE Solving##
###############

tspan = (0., 100000.) #in hours
C₀_ECA = analytical_steady_state_AWB_full_ECA(I_S(0), I_D(0), u_Q_ref, Q, a_MSA, K_DE, K_UE, V_DE_ref, V_UE_ref, Ea_V_DE, Ea_V_UE, r_M, r_E, r_L, temp_ref)
print(C₀_ECA)
p_ECA = [u_Q_ref, Q, a_MSA, K_DE, K_UE, V_DE_ref, V_UE_ref, Ea_V_DE, Ea_V_UE, r_M, r_E, r_L]

#AWB_ECA version
function AWB_ECA!(du, u, p, t)
    S, D, M, E = u
    u_Q_ref, Q, a_MSA, K_DE, K_UE, V_DE_ref, V_UE_ref, Ea_V_DE, Ea_V_UE, r_M, r_E, r_L = p
    du[1] = dS = I_S(t) + a_MSA * r_M * M - ((arrhenius_temp_dep(V_DE_ref, temp_gen(t, temp_ref), Ea_V_DE, temp_ref) * E * S) / (K_DE + E + S))
    du[2] = dD = I_D(t) + (1 - a_MSA) * r_M * M + ((arrhenius_temp_dep(V_DE_ref, temp_gen(t, temp_ref), Ea_V_DE, temp_ref) * E * S) / (K_DE + E + S)) + r_L * E - ((arrhenius_temp_dep(V_UE_ref, temp_gen(t, temp_ref), Ea_V_UE, temp_ref) * M * D) / (K_UE + M + D))
    du[3] = dM = linear_temp_dep(u_Q_ref, temp_gen(t, temp_ref), Q, temp_ref) * ((arrhenius_temp_dep(V_UE_ref, temp_gen(t, temp_ref), Ea_V_UE, temp_ref) * M * D) / (K_UE + M + D)) - r_M * M - r_E * M
    du[4] = dE = r_E * M - r_L * E
end

AWB_ECA_prob = ODEProblem(AWB_ECA!, C₀_ECA, tspan, p_ECA)
AWB_ECA_sol = solve(AWB_ECA_prob, CVODE_BDF(), dt = 0.05, saveat = 0:2:100000) #sample result every 2 hour(s), forcing dt = 0.05 hours

[53.60590075209108, 0.19080779944289697, 0.6707317073170732, 0.013414634146341465]

retcode: Success
Interpolation: 1st order linear
t: 50001-element Vector{Float64}:
      0.0
      2.0
      4.0
      6.0
      8.0
     10.0
     12.0
     14.0
     16.0
     18.0
     20.0
     22.0
     24.0
      ⋮
  99978.0
  99980.0
  99982.0
  99984.0
  99986.0
  99988.0
  99990.0
  99992.0
  99994.0
  99996.0
  99998.0
 100000.0
u: 50001-element Vector{Vector{Float64}}:
 [53.60590075209108, 0.19080779944289697, 0.6707317073170732, 0.013414634146341465]
 [53.6045411069456, 0.19116497327923662, 0.6709001541646021, 0.013414636841881392]
 [53.60172032593861, 0.19202643777272824, 0.6712221228663473, 0.013414644302297567]
 [53.59779760762747, 0.19329315455569177, 0.671653311598881, 0.0134146590820407]
 [53.594086125638135, 0.19443680469027735, 0.6720722829338039, 0.013414681861633792]
 [53.59172376512138, 0.19506314624571416, 0.6723607759910643, 0.013414711101235677]
 [53.59091897284771, 0.1952397206716412, 0.6724667208023151, 0.013414743776974785]
 [53.59077069422485, 0.1954063099

In [5]:
##################
##Exporting Data##
##################

df_AWB_ECA_soln = DataFrame(AWB_ECA_sol.u, :auto)
df_AWB_ECA_soln

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,53.6059,53.6045,53.6017,53.5978,53.5941,53.5917,53.5909,53.5908
2,0.190808,0.191165,0.192026,0.193293,0.194437,0.195063,0.19524,0.195406
3,0.670732,0.6709,0.671222,0.671653,0.672072,0.672361,0.672467,0.672454
4,0.0134146,0.0134146,0.0134146,0.0134147,0.0134147,0.0134147,0.0134147,0.0134148


In [6]:
CSV.write("AWB_ECA_synthetic_soln.csv", df_AWB_ECA_soln)

"AWB_ECA_synthetic_soln.csv"